## Setup

In [1]:
from datasets import Dataset,DatasetDict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoModelForSequenceClassification,AutoTokenizer, AutoConfig
from transformers import TrainingArguments, Trainer
import torch
import os
import random
from transformers import EvalPrediction, TrainerCallback
import torch
from sklearn.metrics import mean_squared_error
import gc
import transformers
import wandb

## Constants

In [2]:
BASE_PATH = "/notebooks"

In [35]:
TEST_PATH = os.path.join(BASE_PATH, 'data/test/test.csv')
MODEL_0_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_0")
MODEL_1_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_1")
MODEL_2_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_2")
MODEL_3_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_3")
MODEL_4_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_4")
MODEL_5_DIR = os.path.join(BASE_PATH, "trainerDEBERTA-LARGE/model_fold_5")
MODELS = [MODEL_0_DIR, MODEL_1_DIR, MODEL_2_DIR, MODEL_3_DIR,MODEL_4_DIR, MODEL_5_DIR]

In [36]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 28
seed_everything(seed=SEED)
MAX_LENGTH = 256
BATCH_SIZE = 4

## Functions

In [37]:
def tok_func(x): return tokenizer(x["full_text"],padding="max_length", truncation=True, max_length = 256)

In [38]:
preds_list = []
for MODEL in MODELS:
    transformers.logging.set_verbosity_error()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_0_DIR)
    df_test = pd.read_csv(TEST_PATH)
    test_dataset = Dataset.from_pandas(df_test)
    test_dataset = test_dataset.map(tok_func, batched=True, remove_columns=["text_id","full_text"])
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_0_DIR)
    # arguments for Trainer
    test_args = TrainingArguments(
        output_dir = MODEL_0_DIR,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = BATCH_SIZE,   
        dataloader_drop_last = False,
    )

    # init trainer
    trainer = Trainer(
                  model = model, 
                  args = test_args,
              )
    preds = trainer.predict(test_dataset)
    preds_list.append(torch.Tensor(preds.predictions))

  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 3
  Batch size = 4


  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 3
  Batch size = 4


  0%|          | 0/1 [00:00<?, ?ba/s]

...

***** Running Prediction *****
  Num examples = 3
  Batch size = 4


  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 3
  Batch size = 4


  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 3
  Batch size = 4


In [39]:
pred_tot = torch.zeros(size = preds_list[0].shape)
for i, pred in enumerate(preds_list):
    pred_tot += pred/len(preds_list)

In [44]:
df_ss = pd.read_csv("data/sample_submission.csv")
sub = pd.DataFrame(pred_tot.numpy())
sub.columns = df_ss.columns[1:]
sub["text_id"] = df_test["text_id"]
sub = sub[df_ss.columns]
sub 

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.834514,2.676057,3.022908,2.875986,2.672467,2.418565
1,000BAD50D026,2.794237,2.687908,2.786851,2.528409,2.319565,2.719576
2,00367BB2546B,3.401883,3.148318,3.381846,3.244108,3.121199,2.986397
